In [9]:
import pandas as pd
import networkx as nx

# read csv files
movements_df = pd.read_csv("/home/vsicard/Documents/POSTDOC-INSERM/DATA/cattle_data//movements.csv")
attributes_df = pd.read_csv("/home/vsicard/Documents/POSTDOC-INSERM/DATA/cattle_data/attributes.csv", low_memory=False)

In [10]:
# Create empty graph
G = nx.MultiGraph()

# create nodes
for _, row in attributes_df.iterrows():
    node = row['node']
    attributes = {
        'ID': row['ID'],
        'species': row['species'],
        'longitude': row['longitude'],
        'latitude': row['latitude'],
        'istat': row['istat'],
        'ptype': row['ptype'],
        'province': row['province'],
        'region': row['region'],
        'region_name': row['region_name'],
        'state': 'S'
    }
    G.add_node(node, **attributes)

In [ ]:
#add edges with attributes
for _, row in movements_df.iterrows():
    source = row['i']
    target = row['j']
    weight = row['w']
    time = row['t']
    G.add_edge(source, target, t=time, weight=weight)
    
print('number of nodes:', G.number_of_nodes())
print('number of edges:', G.number_of_edges())

In [ ]:
import random

# only one t graph

# Extraction du sous-graphe à t = 0
G_t0 = nx.edge_subgraph(G, [(u, v, attr) for u, v, attr in G.edges(data='t', default=0) if attr == 0])
G_t0.nodes = G.nodes

print(G_t0.number_of_edges())

    

In [ ]:
ii = 0
for node in G_t0.nodes:
    if ii < 5:
        print(G_t0.nodes[node]['state'], list(G_t0.neighbors(node)))
    else:
        G_t0.nodes[node]['state'] = 'I'
        print(G_t0.nodes[node]['state'])
        break
    ii += 1

In [7]:
# SIR
max_time = 3
for t in range(max_time):
    for node in G_t0.nodes: 
        state = G_t0.nodes[node]['state']
        if state == 'S':
            if any([G_t0.edges[edge]['t'] == t and G_t0.nodes[node]['state'] == 'I' for edge in G_t0.edges]):
                G_t0.nodes[node]['state'] = 'I'
        elif state == 'I':
            infected_neighbours = [n for n in G_t0.neighbors(node) if G_t0.nodes[n]["states"] == 'I' and any([G_t0.edges[edge]['t'] == t for edge in G_t0.edges])]
            if random.random() < recovery_rate:
                G_t0.nodes[node]['state'] = 'R'

for node in G_t0.nodes:
    if G_t0.nodes[node]['state'] == 'I':
        print(f"Node: {node}, State: {G_t0.nodes[node]['state']}")
    


NetworkXError: The node 19.0 is not in the graph.